In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from timeit import default_timer as timer

In [4]:
from load_data import load_file
from helper_functions import calc_scaling,symmetrize,rotate_to_reference,single_rotD,single_rotP
from calc_average import full_average,full_average_IR,full_average_R,full_average_R_orth,numerical_sector_average
from calc_single import oriented_IR,oriented_R,single_rot_IR,single_rot_R

In [5]:
#filename="freq-19813-90-2.out"   # less accurate
filename="freq-19813-90-2.fchk"   # more accurate
fr,Z,Q,D,P0,nat,aniso=load_file(filename)

Loaded  freq-19813-90-2.fchk


In [6]:
phys_params=dict(laser =785, #633, 
                 T = 298.15
                 )

# calculate intensity scaling factors
v0= math.pow(10, 7)/phys_params['laser']

scalingIR,scaling,scalingexp= calc_scaling(phys_params['T'])
print("IR scaling ",scalingIR)
print("Raman internal scaling ",scaling)
pi=math.pi
torad=2*pi/360

IR scaling  2925.374334329951
Raman internal scaling  2.060252732874952e-45


In [7]:
# modes: list of normal modes to calculate
modes=[31,40]

rotate=0    # rotate molecule to reference orientation before analysis, NOT IMPLEMENTED YET
numerical=1 # perform numerical tests
single=1    # get intensities for single rotations

# rotational angles for single rotations 
nx=4      
ths=np.linspace(0,pi/2,  num=nx) 
phs=np.linspace(0,2*pi,  num=nx)

Lm=1

# Full orientation averages
nummodes=len(modes)
conv_av=np.zeros((nummodes))     # conversion intensity
ir_av=np.zeros((nummodes))    # IR intensity
r_av=np.zeros((nummodes))     # Raman Stokes intensity, parallel fields
r_a_av=np.zeros((nummodes))   # Raman anti-Stokes intensity, parallel fields
r_av_ort=np.zeros((nummodes)) # Raman Stokes intensity, orthogonal fields

# Single orientations
ir_single=np.zeros((nummodes,len(ths),len(phs)))  # IR intensity
r_single=np.zeros((nummodes,len(ths),len(phs)))   # Raman Stokes, parallel fields
conv_single=np.zeros((nummodes,len(ths),len(phs))) # Conversion intensity, parallel fields

for n,m in enumerate(modes):
    print("Mode ",m)
    P=symmetrize(P0[m,:,:])
    if rotate:
        Drot,Prot,R=rotate_to_reference(D[m,:],P,Z,Q)
        
# Calculate frequency-dependent scaling factors
    # Usual Stokes for thermal population
    scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
                fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
    # Usual anti-Stokes for thermal population
    scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
           1/(-1+math.exp(-scalingexp * fr[m]))) # 
    # For THOR: anti-Stokes without population
    scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] 

# Calculate full averages analytically
    starta=timer()
    conv_av[n] = scalingIR *scalingTHOR*full_average(D[m,:],P)  
    ir_av[n]=scalingIR*full_average_IR(D[m,:]) 
    Rav=full_average_R(P)
    r_av[n]=scalingR*Rav 
    enda=timer()
    r_a_av[n]=scalingaR*Rav 
    r_ort,depol=full_average_R_orth(P)
    r_av_ort[n]=scalingR*r_ort 
    
# Test: Calculate full averages numerically 
    if numerical:
            startm=timer()
            ir_numav, r_numav,p_numav=numerical_sector_average(D[m,:],P,1,1,0,0,0)
            ir_numav=scalingIR*ir_numav
            r_numav=scalingR*r_numav
            p_numav=scalingIR *scalingTHOR*p_numav
            endm=timer()
            print("Time for evaluation\n   Analytic \t{:.1e}s \n   Numerical \t{:.1e}s \n   Num/An \t{:.1e}".format(
                enda-starta,endm-startm,(endm-startm)/(enda-starta)))
            print("Abs. and rel. error of numerical results\n   IR \t\t{:.2e}\t{:.2f}% \n   Raman \t{:.2e}\t{:.2f}% \n   Conv \t{:.2e}\t{:.2f}%".format(
                ir_numav-ir_av[n],(ir_numav-ir_av[n])/ir_av[n]*100,r_numav-r_av[n],(r_numav-r_av[n])/r_av[n]*100,
                p_numav-conv_av[n],(p_numav-conv_av[n])/conv_av[n]*100))
            print("-----------------------------------------------")
            
# Calculate single orientations analytically
    if single:
            if numerical:
                print("IR intensity                            | Raman intensity")
                print("single    numeric1  numeric2   orient   | single    numeric1   numeric2   orient")
            for i,th in enumerate(ths):
                for j,ph in enumerate(phs):
                    ir=single_rot_IR(D[m,:],a=ph,b=th,c=0) 
                    r=single_rot_R(P,a=ph,b=th,c=0)
                    ir_single[n,i,j]=scalingIR*ir
                    r_single[n,i,j]=scalingR*r
                    conv_single[n,i,j]=scalingIR*scalingTHOR*ir*r
                    
# Tests: numerical average on rotated D and P should give the same value 
#             when using nump=1, 
#             or in the limit of k->inf and l->inf
#        rotating D and P first and using field oriented along z should also give the same
                    if numerical:
                        Drot=single_rotD(D[m,:],a=ph,b=th,c=0)
                        Prot=single_rotP(P,a=ph,b=th,c=0)
                        ir_snumav, r_snumav,p_snumav=numerical_sector_average(Drot,Prot,1,1,0,0,0,nump=1)
                        ir_snumav2, r_snumav2,p_snumav2=numerical_sector_average(Drot,Prot,1000000,1000000,0,0,0,nump=30)
                        print("{:.3e} {:.3e} {:.3e} {:.3e} | {:.3e} {:.3e} {:.3e} {:.3e}".format(ir_single[n,i,j],
                            scalingIR*ir_snumav,scalingIR*ir_snumav2,scalingIR*oriented_IR(Drot),
                            r_single[n,i,j],
                            scalingR*r_snumav,scalingR*r_snumav2,scalingR*oriented_R(Prot)))

Mode  31
Time for evaluation
   Analytic 	7.6e-05s 
   Numerical 	4.6e-01s 
   Num/An 	6.0e+03
Abs. and rel. error of numerical results
   IR 		-5.99e-03	-0.11% 
   Raman 	-2.92e-34	-0.02% 
   Conv 	-7.84e-35	-0.00%
-----------------------------------------------
IR intensity                            | Raman intensity
single    numeric1  numeric2   orient   | single    numeric1   numeric2   orient
3.642e-02 3.642e-02 3.641e-02 3.642e-02 | 2.652e-33 2.652e-33 2.652e-33 2.652e-33
3.642e-02 3.642e-02 3.641e-02 3.642e-02 | 2.652e-33 2.652e-33 2.652e-33 2.652e-33
3.642e-02 3.642e-02 3.642e-02 3.642e-02 | 2.652e-33 2.652e-33 2.653e-33 2.652e-33
3.642e-02 3.642e-02 3.641e-02 3.642e-02 | 2.652e-33 2.652e-33 2.652e-33 2.652e-33
2.195e+00 2.195e+00 2.195e+00 2.195e+00 | 6.118e-32 6.118e-32 6.118e-32 6.118e-32
3.201e-04 3.201e-04 3.201e-04 3.201e-04 | 7.189e-34 7.189e-34 7.188e-34 7.189e-34
3.980e+00 3.980e+00 3.980e+00 3.980e+00 | 7.389e-31 7.389e-31 7.389e-31 7.389e-31
2.195e+00 2.195e+00 2.1

In [8]:
# Test: check if average of single rotations gives full average
# weighting function: pi*sin(theta)/(2*number of single rotations)
# set nx larger for high accuracy
print("\tfull average  \tav. of single rotations \terror%")
ir_single_av=np.zeros((nummodes))
r_single_av=np.zeros((nummodes))
conv_single_av=np.zeros((nummodes))

for n,m in enumerate(modes):
    print ("Mode ",m)
    ir_single_av[n]=pi*np.sum(np.sum(ir_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
    print("IR  \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(ir_av[n],ir_single_av[n],
                                                      (ir_single_av[n]-ir_av[n])/ir_av[n]))
    
    r_single_av[n]=pi*np.sum(np.sum(r_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
    print("Raman \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(r_av[n],r_single_av[n],
                                                        (r_single_av[n]-r_av[n])/r_av[n]))

    
    conv_single_av[n]=pi*np.sum(np.sum(conv_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
    print("Conv.  \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(
        conv_av[n],conv_single_av[n],(conv_single_av[n]-conv_av[n])/conv_av[n]))

	full average  	av. of single rotations 	error%
Mode  31
IR  	5.429e+00 		6.071e+00 		0.118%
Raman 	1.206e-30 		1.355e-30 		0.124%
Conv.  	2.512e-29 		2.782e-29 		0.108%
Mode  40
IR  	4.200e+00 		3.974e+00 		-0.054%
Raman 	1.062e-30 		1.137e-30 		0.071%
Conv.  	1.791e-29 		1.873e-29 		0.046%
